In [1]:
from importlib import reload
import sys
src_path =   '../src' # change as needed
sys.path.insert(0,src_path)

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import model_utils; reload(model_utils)

max_length=10

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = model_utils.NER()
    
model.generate(max_length, train_layers=0, optimizer = "adam", debias=False)

model_utils.initialize_vars(sess)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 10)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 10)]         0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, None, 768)    108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [2]:
import tensorflow_hub as hub
from bert import tokenization

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module("https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1")
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        with tf.Session(config=config) as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
    return tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

In [21]:
token_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("european"))

ids = token_ids + [0 for i in range(max_length - len(token_ids))]

masks = [1 for i in range(len(token_ids))] + [0 for i in range(max_length - len(token_ids))]

sequenceIDs = [0 for i in range(max_length)]

for result in model.yieldBertEmbeddings([[ids],[masks],[sequenceIDs]]):
    european_vector = np.sum(result, axis=0)

In [22]:
token_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("african"))

ids = token_ids + [0 for i in range(max_length - len(token_ids))]

masks = [1 for i in range(len(token_ids))] + [0 for i in range(max_length - len(token_ids))]

sequenceIDs = [0 for i in range(max_length)]

for result in model.yieldBertEmbeddings([[ids],[masks],[sequenceIDs]]):
    african_vector = np.sum(result, axis=0)

In [23]:
token_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("caucasian"))

ids = token_ids + [0 for i in range(max_length - len(token_ids))]

masks = [1 for i in range(len(token_ids))] + [0 for i in range(max_length - len(token_ids))]

sequenceIDs = [0 for i in range(max_length)]

for result in model.yieldBertEmbeddings([[ids],[masks],[sequenceIDs]]):
    caucasian_vector = np.sum(result, axis=0)

In [27]:
token_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("american"))

ids = token_ids + [0 for i in range(max_length - len(token_ids))]

masks = [1 for i in range(len(token_ids))] + [0 for i in range(max_length - len(token_ids))]

sequenceIDs = [0 for i in range(max_length)]

for result in model.yieldBertEmbeddings([[ids],[masks],[sequenceIDs]]):
    american_vector = np.sum(result, axis=0)

In [29]:
spatial.distance.cosine(american_vector, african_vector)

0.2759936451911926